#### ДЗ №3 по Обнаружению и распознаванию сигналов. Часть 2
Николаева Екатерина ИУ8-81

В Части 1 был реализован интерфейс для сбора данных в БД.
Для классификации постов были выбраны следующие темы:
1) Кулинария
2) Наука
3) Спорт
4) Путешествия
5) Другое

А также следующие признаки для обучения моделей:
1) Название сообщества, в котором опубликован пост
2) Тематика сообщества
3) Статус сообщества
4) Описание сообщества
5) Текст в посте
6) Первое изображение в посте
7) Названия первых 7 обсуждений в группе
8) Аватарка группы
9) Текст в последних 7 записях в сообществе
10) Количество подписчиков сообщества
11) Количество фото в сообществе
12) Количество видео в сообществе
13) Количество аудиозаписей в сообществе
14) Количество статей в сообществе

Была подключена база данных Microsoft SQL Server, в которой существует 2 таблицы: posts - содержит id постов, признаки из постов и классы, которым принадлежат эти посты для обучения моделей; test_posts - содержит id постов и классы для проверки работы обученной модели.

In [11]:
#достаем данные из БД и записываем в массив myData

import pypyodbc

def text_cleaner(text):
    text = text.lower()    
    reg = re.compile('[^а-яА-Я ]')
    res = reg.sub('', text)
    for i in range(len(res)):
        if text[i] == ' ': res = res[i+1:]
        if text[i] != ' ': break 
    return  res 

myServer = 'DESKTOP-UJ2J63A\SQLEXPRESS'
myDatabase = 'VK_POSTS'

connection = pypyodbc.connect('Driver={ODBC Driver 17 for SQL Server};'
                                  'Server=' + myServer + ';'
                                  'Database=' + myDatabase + ';'
                                  'Trusted_Connection=yes;')
cursor = connection.cursor()
mySQLQuery = ("""
                    SELECT * FROM dbo.posts 
              """)

cursor.execute(mySQLQuery)
result = cursor.fetchall()
myData = []
for post in result:
    temp = []
    for info in post:
        temp.append(info)
    myData.append(temp)
cursor.close()
connection.close()

print('Complete')

Complete


Будем обрабатывать текстовые признаки в модели SGDClassifier - линейный классификатор, обучающий модель методом стохастичекого градиентного спуска. У такого алгоритма есть преимущества: высокая скорость обучения и простота реализации модели.
Для каждого поста отдельно берем каждый текстовый признак и обучаем на них модель. Модель будет предсказывать класс для каждого текстового признака в посте, далее будем записывать все полученные из модели классы в массив. Каждому классу присвоен свой уникальный номер (1 элемент в массиве соответсвует классу, 
которому соответствует название группы, 2 элемент - классу статуса группы, 3 - классу описания в группе и т.д. Все числовые признаки записываются в массив без изменений)

In [12]:
# обрабатываем текстовые признаки

import sys
import numpy as np
import pickle
import re
from Stemmer import Stemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
#from sklearn.metrics import accuracy_score
import codecs

# присваиваем классам уникальный номер
def class_to_num(name):
    if name == 'Кулинария':
        return 1
    if name == 'Наука':
        return 2
    if name == 'Путешествия':
        return 3
    if name == 'Спорт':
        return 4
    if name == 'Другое':
        return 5

# загружаем данные для обучения модели    
def load_data():
    data = {'text':[],'tag':[]}
    for j in range(len(loading)):
        for i in range (len(loading[j]) - 1):
            data['text'] += [text_cleaner(loading[j][i])]
            data['tag'] += [loading[j][-1]]      
    return data


def train_test_split(data, validation_split = 0.05):
    n = len(data['text'])
    indices = np.arange(n)
    np.random.shuffle(indices)

    X = [data['text'][i] for i in indices]
    Y = [data['tag'][i] for i in indices]
    nb_validation_samples = int( validation_split * n )

    return { 
        'train': {'x': X[:-nb_validation_samples], 'y': Y[:-nb_validation_samples]},
        'test': {'x': X[-nb_validation_samples:], 'y': Y[-nb_validation_samples:]}
    }

# формируем вектор значений номеров классив для каждого поста 
n = len(myData)
myData_vec = []
for i in range (n):
    myData_vec.append([0,0,0,0,0,0,0,0,0,0,0,0,0,0])
loading = []


for i in range(len(myData)):
    temp = []
    for k in range (1, 9):
        if (k==6) or (k==8): continue
        if myData[i][k] == '': 
            myData_vec[i][k-1] = 0
        else:
            myData_vec[i][k-1] = class_to_num(myData[i][15])
            temp.append(myData[i][k])
    temp.append(myData[i][15])   
    loading.append(temp)   
    for j in range (9,14):
        myData_vec[i][j] = myData[i][j+1]

# обучаем модель SGDClassifier
data = load_data()
D = train_test_split(data)
text_clf = Pipeline([
                    ('tfidf', TfidfVectorizer()),
                    ('clf', SGDClassifier(loss='hinge', max_iter = 1000)),
                    ])
text_clf.fit(D['train']['x'], D['train']['y'])
#predicted = text_clf.predict( D['train']['x'] )
predicted_sgd = text_clf.predict(D['test']['x'])
print(metrics.classification_report(predicted_sgd, D['test']['y']))

              precision    recall  f1-score   support

      Другое       0.89      0.80      0.84        10
   Кулинария       0.75      0.67      0.71         9
       Наука       0.54      0.88      0.67         8
 Путешествия       0.91      0.71      0.80        14
       Спорт       0.57      0.57      0.57         7

    accuracy                           0.73        48
   macro avg       0.73      0.73      0.72        48
weighted avg       0.76      0.73      0.74        48



Получили метрики для модели SGDClassifier.
Также обучим модель KNeighborsClassifier - классификатор методом ближайших соседей и сравним метрики.
Алгоритм такого классификатора состоит из трех последовательных этапов:

1) вычислить расстояние от целевого объекта (который необходимо классифицировать) до каждого из объектов обучающей выборки (уже маркированных каким-либо классом);
2) отобрать k объектов обучающей выборки, расстояния до которых минимальны (на первом этапе k выбирается произвольно, затем итеративно подбирается лучшее значение k на основе точности полученных прогнозов при каждом из выбранных k );
3) получить класс объекта на основе наиболее часто встречающегося среди k ближайших соседей

In [13]:
knb_ppl_clf = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('knb_clf', KNeighborsClassifier(n_neighbors=10))])
knb_ppl_clf.fit(D['train']['x'], D['train']['y'])

predicted_sgd = knb_ppl_clf.predict(D['test']['x'])
print(metrics.classification_report(predicted_sgd, D['test']['y']))

              precision    recall  f1-score   support

      Другое       0.33      0.27      0.30        11
   Кулинария       0.38      1.00      0.55         3
       Наука       0.23      1.00      0.38         3
 Путешествия       0.09      1.00      0.17         1
       Спорт       0.71      0.17      0.27        30

    accuracy                           0.31        48
   macro avg       0.35      0.69      0.33        48
weighted avg       0.56      0.31      0.30        48



Метрики для SGDClassifier оказались лучше, значит, модель более точная. Удобнее всего посмотреть на метрику «macro-avg» (сначала подсчитывается каждая метрика по каждому классу, а потом усредняется). Macro-avg более устойчива к скошенным распределениям классов. Линейный классификатор показал лучший результат на тестовых данных (0,73 против 0,35 у классификатора методом ближайших соседей).

Теперь будем обучать модель RidgeClassifier для обработки векторов, построенных на предыдущем шаге. Классификатор с помощью регрессии Ridge сначала преобразовывает целевые значения в {-1, 1} и затем рассматривает проблему как задачу регрессии.

In [14]:
from sklearn import linear_model 

features = myData_vec
labels = []
for i in range (len(myData)):
    labels.append(myData[i][15])

reg = linear_model.RidgeClassifier()
reg.fit(features, labels) 
print ('Learning complete')

Learning complete


Обучение закончено. Переходим к тестированию модели. Для этого реализуем функции для получения признаков для тестовых постов.

In [15]:
import vk_api

access_token = ''
vk_session = vk_api.VkApi(token=access_token)
vk = vk_session.get_api()


def get_group_name(vk_api, id_gr):
    name = vk.groups.getById(group_id=(id_gr))[0]
    name = name['name']
    return name

def get_group_activity(vk_api, id_gr):
    activity = vk.groups.getById(group_id=(id_gr), fields=['activity'])[0]
    if 'activity' in activity:
        return activity['activity']
    return None

def get_group_status(vk_api, id_gr):
    status = vk.groups.getById(group_id=(id_gr), fields=['status'])[0]
    if 'status' in status:
        return status['status']
    return None

def get_group_description(vk_api, id_gr):
    descr = vk.groups.getById(group_id=(id_gr), fields=['description'])[0]
    if 'description' in descr:
        return descr['description']
    return None

def get_post_text(vk_api, id_post):
    post = vk.wall.getById(posts=id_post)[0]
    if 'text' in post:
        text = post['text']
        return text
    return None

def get_post_photo(vk_api, id_post):
    post = vk.wall.getById(posts=id_post)[0]
    if 'attachments' not in post:
        return None
    attachments = post['attachments']
    photo = None
    for i in range(len(attachments)):
        if attachments[i]['type'] == 'photo':
            photo = attachments[i]['photo']['sizes'][0]['url']
            break
    return photo

def get_topics(vk_api, id_gr):
    topics = vk.board.getTopics(group_id=id_gr)
    if 'items' not in topics:
        return None
    topics = topics['items']
    group_topics = ''
    for i in range(len(topics)):
        if i == 7: break
        group_topics = group_topics + topics[i]['title'] + ', '
    return group_topics

def get_group_avatar(vk_api, id_gr):
    avatar = vk.groups.getById(group_id=id_gr, fields='photo_50')[0]
    avatar = avatar['photo_50']
    return avatar

def get_group_posts(vk_api, id_gr):
    posts = vk.wall.get(owner_id='-' + id_gr)
    posts = posts['items']
    posts_text = ''
    for i in range(len(posts)):
        if i == 7: break
        post = posts[i]
        if 'text' not in post: continue
        posts_text = posts_text + post['text'] + '; '
    return posts_text

def get_followers(vk_api, id_gr):
    followers = vk.groups.getById(group_id=id_gr, fields='members_count')[0]
    followers = followers['members_count']
    return followers

def get_media(vk_api, id_gr):
    videos = 0
    photos = 0
    articles = 0
    audios = 0
    media = vk.groups.getById(group_id=id_gr, fields='counters')[0]
    media = media['counters']
    if 'videos' in media:
        videos = media['videos']
    if 'photos' in media:
        photos = media['photos']
    if 'audios' in media:
        audios = media['audios']
    if 'articles' in media:
        articles = media['articles']
    return photos, videos, audios, articles


def get_post_info (vk_api, id_post):
    info = []
    id_post = '-' + id_post
    id_gr = id_post[1:].split('_')[0]
    info.append(id_post)
    info.append(get_group_name(vk_api, id_gr))
    info.append(get_group_activity(vk_api, id_gr))
    info.append(get_group_status(vk_api, id_gr))
    info.append(get_group_description(vk_api, id_gr))
    info.append(get_post_text(vk_api, id_post))
    info.append(get_post_photo(vk_api, id_post))
    info.append(get_topics(vk_api, id_gr))
    info.append(get_group_avatar(vk_api, id_gr))
    info.append(get_group_posts(vk_api, id_gr))
    info.append(get_followers(vk_api, id_gr))
    info.append(get_media(vk_api, id_gr)[0])
    info.append(get_media(vk_api, id_gr)[1])
    info.append(get_media(vk_api, id_gr)[2])
    info.append(get_media(vk_api, id_gr)[3])
    return info

def get_post_vec(vk_api, z, model):
    your_post = get_post_info(vk_api, z)
    post_vec = [0,0,0,0,0,0,0,0,0,0,0,0,0,0]
    for i in range (1, len(your_post)):
        if (i==6) or (i==8): continue
        if your_post[i] == '': 
            post_vec[i-1] = 0
        else:
            zz=[]
            zz.append(z)
            if model == 'sgd':
                predicted = text_clf.predict(zz) 
            else:
                predicted = knb_ppl_clf.predict(zz)
            post_vec[i-1] = class_to_num(predicted[0])  
        if i>=10:
            post_vec[i-1] = your_post[i]
    return post_vec       

Достаем из базы данных тестовые данные. Прогоняем через модель и сравниваем полученные результаты с корректными.

In [16]:
myServer = 'DESKTOP-UJ2J63A\SQLEXPRESS'
myDatabase = 'VK_POSTS'

connection = pypyodbc.connect('Driver={ODBC Driver 17 for SQL Server};'
                                  'Server=' + myServer + ';'
                                  'Database=' + myDatabase + ';'
                                  'Trusted_Connection=yes;')
cursor = connection.cursor()
mySQLQuery = ("""
                    SELECT * FROM dbo.test_posts 
              """)

cursor.execute(mySQLQuery)
result = cursor.fetchall()
testData = []
for post in result:
    temp = []
    for info in post:
        temp.append(info)
    testData.append(temp)
cursor.close()
connection.close()

#results = []
print("SGD model")
for i in range(len(testData)):
    test_features = get_post_vec(vk_api, testData[i][0], 'sgd')
    result = reg.predict([test_features])
    result = result[0]
    #results.append(result)
    print('id: ', testData[i][0], ' Results: ', result, ' Correct results: ', testData[i][1])
    
print("KNB model")
for i in range(len(testData)):
    test_features = get_post_vec(vk_api, testData[i][0], 'knb')
    result = reg.predict([test_features])
    result = result[0]
    #results.append(result)
    print('id: ', testData[i][0], ' Results: ', result, ' Correct results: ', testData[i][1])    

SGD model
id:  29559271_8442855  Results:  Другое  Correct results:  Наука
id:  200410438_88187  Results:  Спорт  Correct results:  Наука
id:  99353432_380476  Results:  Другое  Correct results:  Наука
id:  36969581_139579  Results:  Другое  Correct results:  Наука
id:  193159072_151430  Results:  Другое  Correct results:  Наука
id:  122149207_2202203  Results:  Другое  Correct results:  Наука
id:  145697337_87508  Results:  Другое  Correct results:  Путешествия
id:  104002410_13689  Results:  Другое  Correct results:  Путешествия
id:  212102293_835  Results:  Другое  Correct results:  Путешествия
id:  19999602_16725  Results:  Другое  Correct results:  Путешествия
id:  31041636_309108  Results:  Путешествия  Correct results:  Путешествия
id:  162599917_1315  Results:  Другое  Correct results:  Путешествия
id:  169252815_740805  Results:  Спорт  Correct results:  Спорт
id:  16468115_2119810  Results:  Другое  Correct results:  Спорт
id:  56561705_898247  Results:  Спорт  Correct result

Теперь можно ввести id поста вручную, выбрать модель и посмотреть, какой результат она выдаст.

In [18]:
z = input("Введите id поста: ")
m = input("Введите модель: ")
post_vec = get_post_vec(vk_api, z, m)

result = reg.predict([post_vec])
print(result[0])

Введите id поста: 194261818_11097
Введите модель: knb
Путешествия
